## Tutorial 2

Implementation of a simple perceptron neural network.

## 1. Activation function

In [1]:
def sigma(x):
    return torch.tanh(x)

def dsigma(x):
    return 1 - sigma(x)**2

## 2. Loss

In [2]:
def loss(x,t):
    return torch.norm(x-t)**2

def dloss(x,t):
    return 2*(x-t)

## 3. Forward and backward passes

In [3]:
from data_MNIST import dlc_practical_prologue as dpp
# Load the data
train_input, train_target, test_input, test_target = dpp.load_data(one_hot_labels=True,normalize= True)
train_target = 0.9*train_target
test_target = 0.9*test_target

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


#### Forward pass

In [4]:
import torch

In [5]:
def forward_pass(w1, b1, w2, b2, x):
    x0 = x.view(-1,1)
    s1 = w1.mm(x0) + b1
    x1 = sigma(s1)
    s2 = w2.mm(x1) + b2
    x2 = sigma(s2)

    return x0, s1, x1, s2, x2

#### Backward pass

In [6]:
def backward_pass(w1, b1, w2, b2, t, x, s1, x1, s2, x2,
                  dl_dw1, dl_db1, dl_dw2, dl_db2):
    
    dl_dx2 = dloss(x2,t.view(-1,1))   # (10,1)
    dl_ds2 = dl_dx2*dsigma(s2)# (10,1)
    dl_dw2 = dl_ds2.mm(x1.view(1,-1)) # (10,50)
    dl_db2 = dl_ds2 # (10,1)
    dl_dx1 = (w2.t()).mm(dl_ds2) # (50,1)
    dl_ds1 = dl_dx1*dsigma(s1) # (50,1)
    dl_dw1 = dl_ds1.mm(x.view(1,-1)) # (50,784) 
    dl_db1 = dl_ds1 # (50,1)
    return dl_dw1, dl_db1, dl_dw2, dl_db2

### Parameters initialization

In [7]:
n_hidden = 50
epsilon = 1e-6
w1 = torch.normal(torch.zeros(n_hidden,train_input.size(1)), epsilon)
b1 = torch.normal(torch.zeros(n_hidden,1), epsilon)
w2 = torch.normal(torch.zeros(10,50), epsilon)
b2 = torch.normal(torch.zeros(10,1), epsilon)

dl_dw1 = torch.empty(w1.size())
dl_db1 = torch.empty(b1.size())
dl_dw2 = torch.empty(w2.size())
dl_db2 = torch.empty(b2.size())

#### SGD

In [8]:
lr = .01    #Learning rate
n_iter = 10000
N = train_input.size(0)
p = test_input.size(0)
for k in range(n_iter):
    i = int(torch.randint(0, 1000, (1,)))
    x = train_input[i]
    t = train_target[i].view(-1,1)
    x0, s1, x1, s2, x2 = forward_pass(w1, b1, w2, b2, x)
    
    dl_dw1, dl_db1, dl_dw2, dl_db2 = backward_pass(w1, b1, w2, b2, t, x0, s1, x1, s2, x2, dl_dw1, dl_db1, dl_dw2, dl_db2)

    weights = [w1, b1, w2, b2]
    gradient = [dl_dw1, dl_db1, dl_dw2, dl_db2]
    w1, b1, w2, b2 = [weights[j]-lr*gradient[j] for j in range(len(weights))]
    
    if k%1000 == 999:
        pred = [torch.argmax(forward_pass(w1, b1, w2, b2,train_input[n])[-1]) for n in range(N)]
        targ = [torch.argmax(train_target[n]) for n in range(N)]
        loss = ([(pred[l] != targ[l])*1 for l in range(N)].count(1)/N)*100
        print(f"Iter {k}: Train_loss {loss} %")
        
        pred_test = [torch.argmax(forward_pass(w1, b1, w2, b2,test_input[n])[-1]) for n in range(p)]
        targ_test = [torch.argmax(test_target[n]) for n in range(p)]
        loss_test = ([(pred_test[l] != targ_test[l])*1 for l in range(p)].count(1)/p)*100 
        print(f"\t : Test_loss {loss_test} %")

Iter 999: Train_loss 27.200000000000003 %
	 : Test_loss 39.300000000000004 %
Iter 1999: Train_loss 13.4 %
	 : Test_loss 26.3 %
Iter 2999: Train_loss 7.9 %
	 : Test_loss 22.8 %
Iter 3999: Train_loss 6.9 %
	 : Test_loss 21.8 %
Iter 4999: Train_loss 5.5 %
	 : Test_loss 17.4 %
Iter 5999: Train_loss 4.2 %
	 : Test_loss 16.5 %
Iter 6999: Train_loss 3.5999999999999996 %
	 : Test_loss 16.5 %
Iter 7999: Train_loss 2.5 %
	 : Test_loss 16.3 %
Iter 8999: Train_loss 2.1 %
	 : Test_loss 16.8 %
Iter 9999: Train_loss 1.7999999999999998 %
	 : Test_loss 16.900000000000002 %


In [9]:
from data_MNIST import dlc_practical_4_embryo_modified as dpe


ModuleNotFoundError: No module named 'dlc_practical_prologue'